## Load the IMDB dataset and create the vocabulary

In [108]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

EMBEDDING_DIM=50
VOCAB_SIZE=20000

# Get IMDB dataset
imdb = IMDB(split='train')

# Load English tokenizer, tagger, parser and NER
tokenizer = get_tokenizer('spacy', language='en')

# build the vocab
counter = Counter()
for i, (label, line) in enumerate(imdb):
    counter.update(tokenizer(line))

ordered_dict = OrderedDict(counter.most_common()[:VOCAB_SIZE])
vocab = vocab(ordered_dict)

# insert special tokens and set default index to 'unknown'
vocab.insert_token('<PAD>', 0)
vocab.insert_token('<UNK>', 1)
vocab.set_default_index(1)

In [109]:
print(vocab)

Vocab()


## Create embedding vectors from GloVe

In [110]:
import torchtext as text

# load glove embeddings
vec = text.vocab.GloVe(name='6B', dim=50)
# create the embedding matrix, a torch tensor in the shape (num_words+1, embedding_dim)
word_emb = vec.get_vecs_by_tokens(vocab.get_itos())

In [111]:
print()

## Build up train/test dataset

In [112]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# transform input text and label to ids
def process_text(text):
    return vocab(tokenizer(text))

label_to_ids = {'pos':0, 'neg':1}

# preprocess a batch of raw data (string-like text/labels) into tensors
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_to_ids[_label])
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    # label must be in the same size as target
    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter, test_iter = IMDB()

# transform datasets iterator into map style so that they can be repeatedly loaded in a new epoch
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_test = int(len(test_dataset) * 0.9)

# Question 1 - 10% of test data is set aside for validation
# I understand the question asked us to split from train dataset but as per the documentation, the IMDb training and testing data are 
# split equally - 50% train, 50% split. To provide sufficient data for training, I am splitting the test dataset into validation dataset instead.
split_test, split_valid = random_split(test_dataset, [num_test, len(test_dataset) - num_test])

train_dataloader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(split_test, batch_size=128,
                             shuffle=True, collate_fn=collate_batch)


## Define the logistic regression model

In [128]:
# logistic model
import torch
import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        
        # Question 2 : Replace the EmbeddingBag using PyTorch builtin functions that does the same job of computing sentence representation by taking average of the word embeddings.
        # print(word_vec.size())
        self.embedding2 = nn.EmbeddingBag(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
        self.embedding = nn.Embedding(*(word_vec.size()), padding_idx=0).from_pretrained(word_vec, freeze=False)
        
                
        # Question 3 : Write a Fully Connected Layer (FC layer) with output size of 100 followed by a non-linear activation e.g., ReLU
        self.fc1 = nn.Linear(embed_dim, 100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100, 1)
        
        self._init_weights()

    def _init_weights(self):
        """Initialize network parameters 
        """
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        # self.fc2.bias.data.zero()

    def forward(self, text, offsets):
        embedded = self.embedding(text)

        # embedding_mean = []
        # for i in range(len(offsets)-1):
        #   mean = torch.mean(embedded[offsets[i]:offsets[i+1]], dim=1)
        #   embedding_mean.append(mean)
        # print(len(embedding_mean))

        embedded = torch.mean(embedded, dim=1, keepdim = True)
        # print(embedded.shape)

        embedded2 = self.embedding2(text,offsets)
        # print(embedded2.shape)
        
        y = self.fc1(embedded2)
        y = self.relu(y)
        y = self.fc2(y)
        out = torch.sigmoid(y)
        
        # Question 4: Use the new model you define in __init__()
        return out

## Define train and test function

In [136]:
import time

def train(train_loader, valid_loader, n = 10):
    model.train()
    total_acc, total_count = 0, 0

    last_epoch_acc = 1000
    no_impov = 0

    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(train_loader):
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text, offsets)
        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update parameters by stepping the optimizer
        optimizer.step()
        total_acc += ((predicted_label > 0.5) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(train_loader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

    # Early stopping
    epoch_acc = validate(valid_loader)
    print('Validation acc:', epoch_acc)

    if epoch_acc > last_epoch_acc:
        no_improv += 1

        if no_improv >= n:
            print('Early stopping!')
            return model
    else:
        print('trigger times: 0')
        trigger_times = 0

    last_epoch_acc = epoch_acc


def validate(dataloader):  
   model.eval()
   total_acc, total_count = 0,0
   with torch.no_grad():
      for idx, (label, text, offsets) in enumerate(dataloader):
          predicted_label = model(text, offsets)
          loss = criterion(predicted_label, label)
          # total_loss += loss.item()
          total_acc += (predicted_label.argmax(1) == label).sum().item()
          total_count += label.size(0)
   return total_acc/total_count


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Train and evaluate the model for several epochs

In [137]:
# Hyperparameters
EPOCHS = 30 # epoch

model = LogisticRegression(word_vec=word_emb, embed_dim=EMBEDDING_DIM).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()
total_accu = None


for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, valid_dataloader, n=10)
    
    # Question 5: Use your validation set to early stop the model. Remember to early stop when the validation accuracy does not improve for continous N number of epochs where N is a hyperparameter. Set N = 10

| epoch   1 |    50/  196 batches | accuracy    0.500
| epoch   1 |   100/  196 batches | accuracy    0.490
| epoch   1 |   150/  196 batches | accuracy    0.511
Validation acc: 60.6304
| epoch   2 |    50/  196 batches | accuracy    0.500
| epoch   2 |   100/  196 batches | accuracy    0.493
| epoch   2 |   150/  196 batches | accuracy    0.497
Validation acc: 60.7504
| epoch   3 |    50/  196 batches | accuracy    0.496
| epoch   3 |   100/  196 batches | accuracy    0.501
| epoch   3 |   150/  196 batches | accuracy    0.503
Validation acc: 60.7024
| epoch   4 |    50/  196 batches | accuracy    0.508
| epoch   4 |   100/  196 batches | accuracy    0.498
| epoch   4 |   150/  196 batches | accuracy    0.492
Validation acc: 60.7744
| epoch   5 |    50/  196 batches | accuracy    0.493
| epoch   5 |   100/  196 batches | accuracy    0.505
| epoch   5 |   150/  196 batches | accuracy    0.496
Validation acc: 60.9184
| epoch   6 |    50/  196 batches | accuracy    0.501
| epoch   6 |   

In [ ]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.2f}%'.format(accu_test))

test accuracy    63.94%
